## Testing OpenAI's new Agents SDK

OpenAI announced on March 11th, 2025 new Responses API and Agents SDK (https://openai.com/index/new-tools-for-building-agents/) 

This notebook aims to test these new products

Main documentation: https://platform.openai.com/docs/guides/agents

In [37]:
import os

from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field
import json

from IPython.display import display, Markdown
from datetime import datetime
import pytz

from openai import OpenAI
client = OpenAI()

from agents import Agent, Runner, ModelSettings, WebSearchTool, function_tool

#### Components

1. Agents

    - Very similar to Responses API - actually built on it.
    - Arguments (important ones):
        - `instructions` - developer/system message. Can also be dynamic with a function that takes as input the context and reads from it to fill a prompt template
        - `tools` - custom and built-in
        - `output_type` - structured output objects
        - `handoff_description` - description of the agent. This is used when the agent is used as a handoff, so that an LLM knows what it does and when to invoke it.
    - Run them via the `Runner` class (async by default)
        - handles e2e flow - no manual tool execution or handoff is needed. it is done once reached a `final_output` return

2. Tools
    - built in (Hosted) tools (web search, file search, computer use)
    - custom tools - `function–tool` decorator - turn functions (well defined!) into JSON schemas
    - agents can also be used as tools - important difference: Handoff: entire convo goes to new agent. Tool: only the output is passed to the new agent to execute it and hands it back

3. Handoffs
    - basically sub-agents (technically tools / functions) that are being called when an agent-to-agent handoff should happen
    - in handoffs, the new agent receives the **entire conversation history**, tools (or agents-as-tools) **only receive the generated output**

4. Output types ~ Structured outputs
    - pydantic Baseclasses passed to agent under `output_type`

5. Context
    - dataclass that is passed to all agents, handoffs, tools
    - everyone can read from it, fill prompt templates, pass parameters

6. Additionals
    1. Guardrails
    2. Lifecycle events (hooks)

#### First tests - very basic


In [6]:
async def main(agent, user_input):
    
    result = await Runner.run(agent, user_input)
    print(result.final_output)

    return result
    
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    #tools = [], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Write a haiku about recursion in programming."

In [7]:
r = await main(agent, user_input)

Code calls itself, loops,  
In endless self-reference—  
Oops, stack overflow!


In [17]:
r.__dict__

{'input': 'Write a haiku about recursion in programming.',
 'new_items': [MessageOutputItem(agent=Agent(name='Funny guy', instructions='You are a helpful assistant that always responds in a humorous manner', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=0.0, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None), raw_item=ResponseOutputMessage(id='msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd', content=[ResponseOutputText(annotations=[], text='Code calls itself, loops,  \nIn endless self-reference—  \nOops, stack overflow!', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')],
 'raw_responses': [ModelResponse(output=[ResponseOutputMessage(id='msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd', content=[Respons

In [19]:
# in case you want to pass the output of this run as input to the next one
r.to_input_list()

[{'content': 'Write a haiku about recursion in programming.', 'role': 'user'},
 {'id': 'msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd',
  'content': [{'annotations': [],
    'text': 'Code calls itself, loops,  \nIn endless self-reference—  \nOops, stack overflow!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [29]:
r2 = await Runner.run(agent, [*r.to_input_list(), {"role": "user", "content": "now translate this to Russian"}])
r2.to_input_list()

[{'content': 'Write a haiku about recursion in programming.', 'role': 'user'},
 {'id': 'msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd',
  'content': [{'annotations': [],
    'text': 'Code calls itself, loops,  \nIn endless self-reference—  \nOops, stack overflow!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user', 'content': 'now translate this to Russian'},
 {'id': 'msg_67d57e9b16e48192bee56a17491bf06b084aa909d0d06bfd',
  'content': [{'annotations': [],
    'text': 'Код сам себя зовёт,  \nВ бесконечной ссылке —  \nОй, переполнение!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

#### Add tools

In [33]:
async def main(agent, user_input):
    
    result = await Runner.run(agent, user_input)
    
    return result
    
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    tools = [WebSearchTool()], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Get the SP500 performance of the past week and summarize it"

In [34]:
r = await main(agent, user_input)

display(Markdown(r.final_output))

Over the past week, the S&P 500 has been on a rollercoaster ride that would make even the bravest thrill-seekers queasy. Despite a valiant 2.1% rally on Friday, March 14, 2025, the index couldn't shake off its fourth consecutive losing week, dipping 2.3% overall. ([apnews.com](https://apnews.com/article/5d8aafc9aa44102b43c554f51f8c85ba?utm_source=openai))

The market's mood swings were largely fueled by President Trump's tariff tango, leaving investors jittery and the S&P 500 waltzing into correction territory on Thursday, March 13, 2025, closing more than 10% below its mid-February peak. ([axios.com](https://www.axios.com/2025/03/13/stock-market-today-recession-tariffs?utm_source=openai))

Adding to the drama, the University of Michigan's consumer sentiment survey revealed that consumers are about as optimistic as a cat in a room full of rocking chairs, with expectations for business conditions hitting a record low. ([investrade.com](https://www.investrade.com/market-review-march-14-2025/?utm_source=openai))

In summary, the S&P 500's performance this week was like a soap opera: full of twists, turns, and a cliffhanger ending.


## Market Turmoil Amid Tariff Concerns:
- [Wall Street rallies to its best day in months, but that's not enough to salvage its losing week](https://apnews.com/article/99a47b4146ef7204ea94e0e5e46e5c26?utm_source=openai)
- [US stocks rebound as government shutdown fears recede](https://www.ft.com/content/aa8ee6d9-28f0-4c87-8aa1-76bfbbe92013?utm_source=openai)
- [S&P 500 selloff hits correction territory amid tariff fears](https://www.axios.com/2025/03/13/stock-market-today-recession-tariffs?utm_source=openai) 

Custom tools

In [ ]:
def pretty_print_agent_run_events(agent_run_response, display_last_step=False):
    
    for e, i in enumerate(agent_run_response.new_items):
        print("\n" + "=" * 60)
        print(f"✨ Step {e+1}: {i.type} ✨")
        print("=" * 60 + "\n")

        if i.type == 'message_output_item':
            #print(f"Output: {i.raw_item.content[0].text}")
            print("📝 Output:")
            if display_last_step:
                display(Markdown(i.raw_item.content[0].text))
            else:
                print(i.raw_item.content[0].text)
        
        else:
            print("📝 Output:")
            print(f"{i.raw_item}")

        print("\n")

In [39]:
@function_tool  
async def fetch_current_time(location: str) -> str:
    
    """Fetch the current time for a given location.

    Args:
        location: The location to fetch the current time for. Example: Europe/Budapest. Format is pytz timezone string.
    """
    
    _tz = pytz.timezone(location)
    _time = datetime.now(_tz)

    return f"Current time in {location}: {_time.strftime('%Y-%m-%d %H:%M:%S %Z')}"

In [45]:
print(f"Name: {fetch_current_time.name}")
print(f"Description: {fetch_current_time.description}")
print(f"Parameters: \n{json.dumps(fetch_current_time.params_json_schema, indent=2)}")

Name: fetch_current_time
Description: Fetch the current time for a given location.
Parameters: 
{
  "properties": {
    "location": {
      "description": "The location to fetch the current time for. Example: Europe/Budapest. Format is pytz timezone string.",
      "title": "Location",
      "type": "string"
    }
  },
  "required": [
    "location"
  ],
  "title": "fetch_current_time_args",
  "type": "object",
  "additionalProperties": false
}


In [46]:
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    tools = [WebSearchTool(), fetch_current_time], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Check the current time in San Francisco, see if anything interesting is planned there in the next 5 hours"

In [47]:
r = await main(agent, user_input)
display(Markdown(r.final_output))

As of 6:43 AM PDT on Saturday, March 15, 2025, in San Francisco, the city is buzzing with events to kick off your day. Here are some highlights happening within the next five hours:



**St. Patrick's Day Parade**  
Starting at 10:00 AM, the 174th San Francisco St. Patrick's Day Parade marches from 2nd and Market Streets to Civic Center Plaza. Expect vibrant floats, lively music, and a sea of green. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))




**Noe Valley Celtic Festival**  
From 9:00 AM to 7:00 PM, enjoy live music, Irish dancers, food, drinks, and arts and crafts at the Noe Valley Celtic Festival. It's a family-friendly event celebrating Celtic culture. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Free Museum Admission**  
Bay Area residents can enjoy free general admission to the Legion of Honor and de Young Museum starting at 9:30 AM. Explore art collections without spending a dime. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Fiber Fest at the Museum of Craft and Design**  
From 1:00 PM to 5:00 PM, engage in a celebration of creativity with fiber arts, showcasing local artists and interactive exhibits. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Arbor Day Run Green 5K/10K/13.1**  
Celebrate Arbor Day with a fun and eco-friendly run at Crissy Field. The event starts at 7:30 AM, promoting a greener future. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))


Whether you're in the mood for parades, cultural festivals, art, or a refreshing run, San Francisco has something to offer this morning. Enjoy your day! 

In [102]:
pretty_print_agent_run_events(r, True)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d583e532308192b9c51ac1d23fab11093001810b203790', arguments='{"location":"America/Los_Angeles"}', call_id='call_2CGjfDpSHpL7B3R03QBm1wue', name='fetch_current_time', type='function_call', status='completed')



✨ Step 2: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue', 'output': 'Current time in America/Los_Angeles: 2025-03-15 06:43:01 PDT', 'type': 'function_call_output'}



✨ Step 3: tool_call_item ✨

📝 Output:
ResponseFunctionWebSearch(id='ws_67d583e644a481929d58bca4e6af395a093001810b203790', status='completed', type='web_search_call')



✨ Step 4: message_output_item ✨

📝 Output:


As of 6:43 AM PDT on Saturday, March 15, 2025, in San Francisco, the city is buzzing with events to kick off your day. Here are some highlights happening within the next five hours:



**St. Patrick's Day Parade**  
Starting at 10:00 AM, the 174th San Francisco St. Patrick's Day Parade marches from 2nd and Market Streets to Civic Center Plaza. Expect vibrant floats, lively music, and a sea of green. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))




**Noe Valley Celtic Festival**  
From 9:00 AM to 7:00 PM, enjoy live music, Irish dancers, food, drinks, and arts and crafts at the Noe Valley Celtic Festival. It's a family-friendly event celebrating Celtic culture. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Free Museum Admission**  
Bay Area residents can enjoy free general admission to the Legion of Honor and de Young Museum starting at 9:30 AM. Explore art collections without spending a dime. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Fiber Fest at the Museum of Craft and Design**  
From 1:00 PM to 5:00 PM, engage in a celebration of creativity with fiber arts, showcasing local artists and interactive exhibits. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Arbor Day Run Green 5K/10K/13.1**  
Celebrate Arbor Day with a fun and eco-friendly run at Crissy Field. The event starts at 7:30 AM, promoting a greener future. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))


Whether you're in the mood for parades, cultural festivals, art, or a refreshing run, San Francisco has something to offer this morning. Enjoy your day! 

In [103]:
r.to_input_list()

[{'content': 'Check the current time in San Francisco, see if anything interesting is planned there in the next 5 hours',
  'role': 'user'},
 {'id': 'fc_67d583e532308192b9c51ac1d23fab11093001810b203790',
  'arguments': '{"location":"America/Los_Angeles"}',
  'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue',
  'name': 'fetch_current_time',
  'type': 'function_call',
  'status': 'completed'},
 {'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue',
  'output': 'Current time in America/Los_Angeles: 2025-03-15 06:43:01 PDT',
  'type': 'function_call_output'},
 {'id': 'ws_67d583e644a481929d58bca4e6af395a093001810b203790',
  'status': 'completed',
  'type': 'web_search_call'},
 {'id': 'msg_67d583e86a308192acb8956b9673c524093001810b203790',
  'content': [{'annotations': [{'end_index': 506,
      'start_index': 403,
      'title': 'Popular Events in San Francisco in March 2025(March Updated) | Trip.com',
      'type': 'url_citation',
      'url': 'https://hk.trip.com/events/1466914-2025-03-san-francisco-collec

Agents-as-tools

In [104]:
russian_agent = Agent(
    name="Russian agent",
    instructions="You translate the user's message to Russian",
)

hungarian_agent = Agent(
    name="Hungarian agent",
    instructions="You translate the user's message to Hungarian",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions="You are a helpful assistant",
    tools=[
        russian_agent.as_tool(
            tool_name="translate_to_russian",
            tool_description="Translate the user's message to Russian",
        ),
        hungarian_agent.as_tool(
            tool_name="translate_to_hungarian",
            tool_description="Translate the user's message to Hungarian",
        ),
        fetch_current_time
    ],
)

user_input = "Check the current time in San Francisco, then in London. Report back in english, then in hungarian, then in russian!"

o = await Runner.run(orchestrator_agent, input=user_input)
print(o.final_output)

### English
Current time in San Francisco is 07:05 AM on March 15, 2025, and in London, it is 2:05 PM on March 15, 2025.

### Hungarian
San Franciscóban 2025. március 15-én, 07:05 van, míg Londonban 2025. március 15-én, 14:05 órát mutat az idő.

### Russian
Текущее время в Сан-Франциско 07:05 утра 15 марта 2025 года, а в Лондоне 14:05 15 марта 2025 года.


In [105]:
pretty_print_agent_run_events(o, True)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d58917b04c8192a3c4a977740337cc06a1d7095c0ce155', arguments='{"location":"America/Los_Angeles"}', call_id='call_jqiDflzmcjxBKd9FAQLPFpoo', name='fetch_current_time', type='function_call', status='completed')



✨ Step 2: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d58917bfac8192a8362885b85d05c806a1d7095c0ce155', arguments='{"location":"Europe/London"}', call_id='call_e9x8I90u8zwkwFd1js2DFyvB', name='fetch_current_time', type='function_call', status='completed')



✨ Step 3: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_jqiDflzmcjxBKd9FAQLPFpoo', 'output': 'Current time in America/Los_Angeles: 2025-03-15 07:05:11 PDT', 'type': 'function_call_output'}



✨ Step 4: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_e9x8I90u8zwkwFd1js2DFyvB', 'output': 'Current time in Europe/London: 2025-03-15 14:05:11 GMT', 'type': 'function_call_output'}



✨ Step 5: tool_call_item ✨

📝 Output:
ResponseFun

### English
Current time in San Francisco is 07:05 AM on March 15, 2025, and in London, it is 2:05 PM on March 15, 2025.

### Hungarian
San Franciscóban 2025. március 15-én, 07:05 van, míg Londonban 2025. március 15-én, 14:05 órát mutat az idő.

### Russian
Текущее время в Сан-Франциско 07:05 утра 15 марта 2025 года, а в Лондоне 14:05 15 марта 2025 года.